# Senior Rankings

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/wca-ipy/

## Initialisation

Basic approach to determine the project directory

In [524]:
import os, sys

projdir = os.path.realpath(os.path.join(sys.path[0], '..'))

try:
    debug = os.environ['LOGIQX_DEBUG']
except:
    debug = 0

## Generic Class

Generic class to ensure that all custom classes are printable

In [525]:
class Printable:
    def __repr__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

    def __str__(self):
        return str(self.__class__) + ": " + str(self.__dict__)

## Load Extracts

Read CSVs into memory as actual objects

In [526]:
import csv

def readCsv(extractName, classType):
    objects = []

    fn = os.path.join(projdir, 'data', 'extract', '%s.csv' % extractName)
    with open(fn, 'r') as f:
        csvReader = csv.reader(f)

        for inputRow in csvReader:
            objects.append(classType(inputRow))

        return objects

In [527]:
class WcaEvent(Printable):
    def __init__(self, fields):
        self.id, self.name, self.rank, self.format = fields
        self.rank = int(self.rank)
        self.rankings = []

events = {}
for event in readCsv('events', WcaEvent):
    events[event.id] = event

In [528]:
class Competition(Printable):
    def __init__(self, fields):
        self.id, self.name, self.cityName, self.country, self.external_website, self.start_date, self.end_date = fields

competitions = {}
for competition in readCsv('competitions', Competition):
    competitions[competition.id] = competition

In [529]:
class Continent(Printable):
    def __init__(self, fields):
        self.id, self.name = fields

continents = {}
for continent in readCsv('continents', Continent):
    continents[continent.id] = continent

In [530]:
class Country(Printable):
    def __init__(self, fields):
        self.id, self.name, self.continent = fields

countries = {}
for country in readCsv('countries', Country):
    countries[country.id] = country

In [531]:
class Person(Printable):
    def __init__(self, fields):
        self.id, self.name, self.country, self.username, self.usernum, self.age, self.hidden, self.userId = fields
        self.usernum = int(self.usernum)
        self.age = int(self.age)
        self.userId = int(self.userId)
        self.events = []

persons = {}
for person in readCsv('seniors', Person):
    persons[person.id] = person

persons['FAKE_RANGE'] = Person(['FAKE_RANGE', 'FAKE (RANGE)', 'XE', '', 0, 0, 'y', 0])
persons['FAKE_EXACT'] = Person(['FAKE_EXACT', 'FAKE (EXACT)', 'XE', '', 0, 0, 'y', 0])

In [532]:
class Ranking(Printable):
    def __init__(self, fields):
        self.eventId, self.type, self.age, self.personId, self.best, self.rank, self.competitionId, self.ageAtComp = fields
        self.age = int(self.age)
        self.best = int(self.best)
        self.rank = int(self.rank)
        self.ageAtComp = int(self.ageAtComp)

rankings = readCsv('senior_rankings', Ranking)

## Process Rankings

Process the rankings prior to JSON export

In [533]:
def formatResult(event, resultType, result):
    '''Intelligently convert result to appropriate format - e.g. HH:MM:SS.SS'''

    def formatTime(seconds):
        if seconds >= 3600:
            formattedTime = str(seconds // 3600) + ':' + str(seconds % 3600 // 60).zfill(2) + ':' + str(seconds % 60).zfill(2)
        elif seconds >= 60:
            formattedTime = str(seconds // 60) + ':' + str(seconds % 60).zfill(2)
        else:
            formattedTime = str(seconds)
        return formattedTime

    result = int(result)

    if event.format == 'time':
        seconds = result // 100
        formattedResult = formatTime(seconds)

        if not (event == '333fm' and resultType == 'single'):
            centiseconds = result % 100
            formattedResult += "." + str(centiseconds).zfill(2)

    elif event.format == 'multi':
        difference = 99 - result // 10000000
        seconds = result % 10000000 // 100
        missed = result % 100
        solved = difference + missed
        attempted = solved + missed

        formattedResult = '%d/%d in %s' % (solved, attempted, formatTime(seconds))

    else:
        formattedResult = str(result // 100)

        if not (event == '333fm' and resultType == 'single'):
            formattedResult += "." + str(result % 100).zfill(2)

    return formattedResult

In [534]:
class Ranks(Printable):
    def __init__(self, ranking):
        self.type = ranking.type
        self.age = ranking.age
        self.ranks = []
        self.estimate = 0

class Rank(Printable):
    def __init__(self, age, ranking):
        self.rank = ranking.rank
        self.id = ranking.personId
        self.best = formatResult(events[ranking.eventId], ranking.type, ranking.best)
        if ranking.ageAtComp > age:
            self.age = ranking.ageAtComp

In [535]:
# Simplify the loop
prevRanking = rankings[0]
currRanks = Ranks(prevRanking)

# Process rankings
for ranking in rankings:
    if ranking.eventId != prevRanking.eventId or ranking.type != prevRanking.type or ranking.age != prevRanking.age:
        if currRanks and prevRanking.eventId in events:
            events[prevRanking.eventId].rankings.append(currRanks)
        currRanks = Ranks(ranking)

    if ranking.eventId in events:
        if debug or persons[ranking.personId].hidden == 'n':
            currRank = Rank(currRanks.age, ranking)
            currRanks.ranks.append(currRank)

        if ranking.eventId not in persons[ranking.personId].events:
            persons[ranking.personId].events.append(ranking.eventId)

        currRanks.estimate += 1

    prevRanking = ranking

# Store the final ranking
if currRanks and prevRanking.eventId in events:
    events[prevRanking.eventId].rankings.append(currRanks)

## Prepare lists for JSON export

In [536]:
eventsList = events.values()
eventsList = sorted(eventsList, key=lambda e: e.rank)

personsList = persons.values()
personsList = sorted(personsList, key=lambda s: s.name)

countriesList = countries.values()
countriesList = sorted(countriesList, key=lambda c: c.name)

continentsList = continents.values()
continentsList = sorted(continentsList, key=lambda c: c.name)

#competitionsList = competitions.values()
#competitionsList = sorted(competitionsList, key=lambda c: c.name)

## Remove clutter for JSON export

In [537]:
# Remove rank from events
for event in eventsList:
    del(event.rank)

# Remove hidden persons
for person in reversed(personsList):
    if person.hidden == 'y':
        if debug:
            person.name = '* %s *' % person.name
        else:
            personsList.remove(person)

# Remove redundant information from persons
for person in personsList:
    if person.usernum <= 0:
        del(person.username)
        del(person.usernum)

    del(person.hidden)
    del(person.userId)

## Write JSON

Write the final output to a JSON file

In [538]:
import datetime

refreshed = datetime.datetime.now().replace(microsecond=0).isoformat().replace('T', ' ')

import json

jsonDump = 'rankings =' + os.linesep
jsonDump += json.dumps( {
                            'refreshed': refreshed,
                            'events': eventsList,
                            'persons': personsList,
                            'countries': countriesList,
                            'continents': continentsList
                        },
                        default=lambda o: o.__dict__, ensure_ascii=False, indent=0)

fn = os.path.join('..', 'docs', 'data', 'Senior_Rankings.js')

if not os.path.exists(os.path.dirname(fn)):
    os.makedirs(os.path.dirname(fn))

with open(fn, 'w') as f:
    f.write(jsonDump)

In [539]:
print('Senior Rankings updated!')

Senior Rankings updated!


## All Done!